In [9]:
from pandas import read_csv
path='https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
data = read_csv(path, header=None)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


In [15]:
df=data.values
x=df[: , :-1]
y=df[: , -1]
print(x.shape,y.shape)

(178, 13) (178,)


In [17]:
from numpy import mean
from numpy import std
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear')
cv= RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
scores = cross_val_score(model,x,y,scoring='accuracy',cv=cv,n_jobs=-1)
print('Accuracy: %.3f (%.3f)'%(mean(scores) , std(scores)))

Accuracy: 0.953 (0.048)


In [19]:
from sklearn import pipeline
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,QuantileTransformer,KBinsDiscretizer
from sklearn.decomposition import PCA, TruncatedSVD

transforms = list()
transforms.append(('mms',MinMaxScaler()))
transforms.append(('ss',StandardScaler()))
transforms.append(('rs',RobustScaler()))
transforms.append(('qt',QuantileTransformer(n_quantiles=100,output_distribution='normal')))
transforms.append(('kbd',KBinsDiscretizer(n_bins=10,encode='ordinal',strategy='uniform')))
transforms.append(('pca',PCA(n_components=7)))
transforms.append(('svd',TruncatedSVD(n_components=7)))

fu=FeatureUnion(transforms)
model = LogisticRegression(solver='liblinear')
steps=list()
steps.append(('fu',fu))
steps.append(('m',model))
pipeline = Pipeline(steps=steps)

cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

scores = cross_val_score(pipeline,x,y, scoring='accuracy', cv=cv, n_jobs=-1)

print('Accuracy: %.3f (%.3f)'%(mean(scores) , std(scores)))



Accuracy: 0.968 (0.037)


In [21]:
from sklearn.feature_selection import RFE

transforms = list()
transforms.append(('mms',MinMaxScaler()))
transforms.append(('ss',StandardScaler()))
transforms.append(('rs',RobustScaler()))
transforms.append(('qt',QuantileTransformer(n_quantiles=100,output_distribution='normal')))
transforms.append(('kbd',KBinsDiscretizer(n_bins=10,encode='ordinal',strategy='uniform')))
transforms.append(('pca',PCA(n_components=7)))
transforms.append(('svd',TruncatedSVD(n_components=7)))

fu=FeatureUnion(transforms)
rfe=RFE(estimator=LogisticRegression(solver='liblinear'),n_features_to_select=15)
model = LogisticRegression(solver='liblinear')
steps=list()
steps.append(('fu',fu))
steps.append(('rfe',rfe))
steps.append(('m',model))
pipeline = Pipeline(steps=steps)

cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

scores = cross_val_score(pipeline,x,y, scoring='accuracy', cv=cv, n_jobs=-1)

print('Accuracy: %.3f (%.3f)'%(mean(scores) , std(scores)))


Accuracy: 0.989 (0.022)
